In [11]:
import pandas as pd
from tabulate import tabulate
csv_path_schools = "/Users/kaylabiddle/Downloads/Starter_Code-4/PyCitySchools/Resources/schools_complete.csv"
school_data = pd.read_csv(csv_path_schools)
csv_path_students = "/Users/kaylabiddle/Downloads/Starter_Code-4/PyCitySchools/Resources/students_complete.csv"
student_data = pd.read_csv(csv_path_students)

school_data.head()
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [12]:
# Number of unique schools
def calculate_total_schools(df):
    total_schools = df["school_name"].nunique()
    return total_schools

total_schools = calculate_total_schools(school_data)
print(f"Total Number of Schools: {total_schools}")

# Number of Students
def calculate_total_students(df):
    total_students = df["student_name"].nunique()
    return total_students

total_students = calculate_total_students(student_data)
print(f"Total Students: {total_students}")

# Calculated Budget
def calculate_total_budget(df):
    total_budget = df["budget"].sum()
    return total_budget

total_budget = calculate_total_budget(school_data)
print(f"Budget: ${total_budget:,.2f}")


Total Number of Schools: 15
Total Students: 32715
Budget: $24,649,428.00


Student Scores:

In [13]:
# Average math score
def calculate_math_score(df):
    total_math_scores = df["math_score"].sum()
    average_math_score = total_math_scores/total_students
    return average_math_score

average_math_score = calculate_math_score(student_data)
print(f"Average math score: {average_math_score:.2f}%")

# Average reading score
def calculate_reading_score(df):
    total_read_scores = df["reading_score"].sum()
    average_read_score = total_read_scores/total_students
    return average_read_score

average_read_score = calculate_reading_score(student_data)
print(f"Average reading score: {average_read_score:.2f}%")

def calculate_math_passing(df):
    passing_score = 70
    num_passed_math = df[df['math_score'] >= passing_score].shape[0]
    total_students = df.shape[0]
    percent_passed_math = (num_passed_math/total_students)*100
    return percent_passed_math

percent_passed_math = calculate_math_passing(student_data)
print(f"Passing math: {percent_passed_math:.2f}%")

def calculate_reading_passing(df):
    passing_score = 70
    num_passed_reading = df[df['reading_score'] >= passing_score].shape[0]
    total_students = df.shape[0]
    percent_passed_reading = (num_passed_reading/total_students)*100
    return percent_passed_reading

percent_passed_reading = calculate_reading_passing(student_data)
print(f"Passing reading: {percent_passed_reading:.2f}%")

def calculate_both_passing(df):
    passing_score = 70
    num_passed_both = df[(df['reading_score'] >= passing_score) & (df['math_score'] >= passing_score)].shape[0]
    total_students = df.shape[0]
    percent_passed_both = (num_passed_both/total_students)*100
    return percent_passed_both

percent_passed_both = calculate_both_passing(student_data)
print(f"Overall passing: {percent_passed_both:.2f}%")

Average math score: 94.57%
Average reading score: 98.03%
Passing math: 74.98%
Passing reading: 85.81%
Overall passing: 65.17%


In [14]:
merged_data = pd.merge(student_data, school_data, on="school_name")

total_students = merged_data['school_name'].value_counts()

# Total school budget
total_budget = merged_data.groupby('school_name')['budget'].mean()

# Per student budget
per_student_budget = total_budget / total_students

# Average math score
average_math_score = merged_data.groupby('school_name')['math_score'].mean()

# Average reading score
average_reading_score = merged_data.groupby('school_name')['reading_score'].mean()

# Define passing scores
passing_score = 70

# Calculate passing columns
merged_data['passed_math'] = merged_data['math_score'] >= passing_score
merged_data['passed_reading'] = merged_data['reading_score'] >= passing_score
merged_data['overall_passing'] = merged_data['passed_math'] & merged_data['passed_reading']


# % Passing Math
percent_passing_math = merged_data.groupby('school_name')['passed_math'].mean() * 100

# % Passing Reading
percent_passing_reading = merged_data.groupby('school_name')['passed_reading'].mean() * 100

# % Overall Passing
percent_overall_passing = merged_data.groupby('school_name')['overall_passing'].mean() * 100

# Create the summary DataFrame
per_school_summary = pd.DataFrame({
    'School Type': school_data.set_index('school_name')['type'],
    'Total Students': total_students,
    'Total School Budget': total_budget.map('${:,.2f}'.format),
    'Per Student Budget': per_student_budget.map('${:,.2f}'.format),
    'Average Math Score': average_math_score.map('{:.2f}%'.format),
    'Average Reading Score': average_reading_score.map('{:.2f}%'.format),
    '% Passing Math': percent_passing_math.map('{:.2f}%'.format),
    '% Passing Reading': percent_passing_reading.map('{:.2f}%'.format),
    '% Overall Passing': percent_overall_passing.map('{:.2f}%'.format)
})

# Display the summary DataFrame
print(tabulate(school_summary, headers='keys', tablefmt='fancy_grid'))

╒═══════════════════════╤═══════════════╤══════════════════╤═══════════════════════╤══════════════════════╤══════════════════════╤═════════════════════════╤══════════════════╤═════════════════════╤═════════════════════╤════════════════════╕
│                       │ School Type   │   Total Students │ Total School Budget   │ Per Student Budget   │ Average Math Score   │ Average Reading Score   │ % Passing Math   │ % Passing Reading   │ % Overall Passing   │ School Size        │
╞═══════════════════════╪═══════════════╪══════════════════╪═══════════════════════╪══════════════════════╪══════════════════════╪═════════════════════════╪══════════════════╪═════════════════════╪═════════════════════╪════════════════════╡
│ Bailey High School    │ District      │             4976 │ $3,124,928.00         │ $628.00              │ 77.05%               │ 81.03%                  │ 66.68%           │ 81.93%              │ 54.64%              │ Large (2000-5000)  │
├───────────────────────┼───────────

In [15]:
# Top 5 Passing Schools
passing_score = 70
merged_data = pd.merge(student_data, school_data, on="school_name")
merged_data['passed_math'] = merged_data['math_score'] >= passing_score
merged_data['passed_reading'] = merged_data['reading_score'] >= passing_score
merged_data['overall_passing'] = merged_data['passed_math'] & merged_data['passed_reading']

school_passing_summary = merged_data.groupby('school_name').agg({
    'overall_passing': 'mean',
    'Student ID': 'count'
}).rename(columns={'overall_passing': 'overall_passing_rate', 'Student ID': 'total_students'})

school_passing_summary['overall_passing_rate'] = school_passing_summary['overall_passing_rate'] * 100

sorted_schools = school_passing_summary.sort_values(by='overall_passing_rate', ascending=False)

top_schools = sorted_schools.head(5)

print(f"\033[1mHighest-Preforming:\033[0m")
print(tabulate(top_schools, headers='keys', tablefmt='fancy_grid'))

# Worst 5 schools
sorted_schools_ascending = school_passing_summary.sort_values(by='overall_passing_rate', ascending=True)

bottom_schools = sorted_schools_ascending.head(5)

print(f"\033[1mLowest-Preforming:\033[0m") 
print(tabulate(bottom_schools, headers='keys', tablefmt='fancy_grid'))


Highest-Preforming:
╒═════════════════════╤════════════════════════╤══════════════════╕
│ school_name         │   overall_passing_rate │   total_students │
╞═════════════════════╪════════════════════════╪══════════════════╡
│ Cabrera High School │                91.3348 │             1858 │
├─────────────────────┼────────────────────────┼──────────────────┤
│ Thomas High School  │                90.948  │             1635 │
├─────────────────────┼────────────────────────┼──────────────────┤
│ Griffin High School │                90.5995 │             1468 │
├─────────────────────┼────────────────────────┼──────────────────┤
│ Wilson High School  │                90.5826 │             2283 │
├─────────────────────┼────────────────────────┼──────────────────┤
│ Pena High School    │                90.5405 │              962 │
╘═════════════════════╧════════════════════════╧══════════════════╛
Lowest-Preforming:
╒═══════════════════════╤════════════════════════╤══════════════════╕
│ schoo

In [16]:
# Average Score by Grade

# Calculate average math scores by school and grade
average_math_scores_by_school_grade = student_data.groupby(['school_name', 'grade']).agg({'math_score': 'mean'}).unstack()
average_math_scores_by_school_grade.columns = average_math_scores_by_school_grade.columns.droplevel()
average_math_scores_by_school_grade = average_math_scores_by_school_grade[['9th', '10th', '11th', '12th']]
average_math_scores_by_school_grade = average_math_scores_by_school_grade.rename(columns={'9th': '9th Grade', '10th': '10th Grade', '11th': '11th Grade', '12th': '12th Grade'})

# Calculate average reading scores by school and grade
average_reading_scores_by_school_grade = student_data.groupby(['school_name', 'grade']).agg({'reading_score': 'mean'}).unstack()
average_reading_scores_by_school_grade.columns = average_reading_scores_by_school_grade.columns.droplevel()
average_reading_scores_by_school_grade = average_reading_scores_by_school_grade[['9th', '10th', '11th', '12th']]
average_reading_scores_by_school_grade = average_reading_scores_by_school_grade.rename(columns={'9th': '9th Grade', '10th': '10th Grade', '11th': '11th Grade', '12th': '12th Grade'})

# Print the results using tabulate
print(f"\033[1mMath Scores By Grade and School:\033[0m")
print(tabulate(average_math_scores_by_school_grade, headers='keys', tablefmt='fancy_grid'))

print(f"\033[1mReading Scores By Grade and School:\033[0m")
print(tabulate(average_reading_scores_by_school_grade, headers='keys', tablefmt='fancy_grid'))


Math Scores By Grade and School:
╒═══════════════════════╤═════════════╤══════════════╤══════════════╤══════════════╕
│ school_name           │   9th Grade │   10th Grade │   11th Grade │   12th Grade │
╞═══════════════════════╪═════════════╪══════════════╪══════════════╪══════════════╡
│ Bailey High School    │     77.0837 │      76.9968 │      77.5156 │      76.4922 │
├───────────────────────┼─────────────┼──────────────┼──────────────┼──────────────┤
│ Cabrera High School   │     83.0947 │      83.1545 │      82.7656 │      83.2775 │
├───────────────────────┼─────────────┼──────────────┼──────────────┼──────────────┤
│ Figueroa High School  │     76.403  │      76.54   │      76.8843 │      77.1514 │
├───────────────────────┼─────────────┼──────────────┼──────────────┼──────────────┤
│ Ford High School      │     77.3613 │      77.6723 │      76.9181 │      76.18   │
├───────────────────────┼─────────────┼──────────────┼──────────────┼──────────────┤
│ Griffin High School   │     82

In [17]:
# Grades based off of spending
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
merged_data['spending_range'] = pd.cut(merged_data['budget'] / merged_data['size'], bins=spending_bins, labels=labels)
passing_score = 70

spending_summary = merged_data.groupby('spending_range').agg({
    'math_score': 'mean',
    'reading_score': 'mean',
    'Student ID': 'count',
    'passed_math': 'mean',
    'passed_reading': 'mean',
    'overall_passing': 'mean'

}).rename(columns={
    'math_score': 'average_math_score',
    'reading_score': 'average_reading_score',
    'Student ID': 'total_students',
    'passed_math': 'percent_passing_math',
    'passed_reading': 'percent_passing_reading',
    'overall_passing': 'percent_overall_passing'
})
spending_summary['percent_passing_math'] *= 100
spending_summary['percent_passing_reading'] *= 100
spending_summary['percent_overall_passing'] *= 100

print(f"\033[1mScores by School Spening:\033[0m")
print(tabulate(spending_summary, headers='keys', tablefmt='fancy_grid'))

Scores by School Spening:
╒══════════════════╤══════════════════════╤═════════════════════════╤══════════════════╤════════════════════════╤═══════════════════════════╤═══════════════════════════╕
│ spending_range   │   average_math_score │   average_reading_score │   total_students │   percent_passing_math │   percent_passing_reading │   percent_overall_passing │
╞══════════════════╪══════════════════════╪═════════════════════════╪══════════════════╪════════════════════════╪═══════════════════════════╪═══════════════════════════╡
│ <$585            │              83.3631 │                 83.964  │             6368 │                93.7029 │                   96.6866 │                   90.6407 │
├──────────────────┼──────────────────────┼─────────────────────────┼──────────────────┼────────────────────────┼───────────────────────────┼───────────────────────────┤
│ $585-630         │              79.9829 │                 82.3126 │             9167 │                79.1099 │           

In [22]:
# Define bins and labels for school size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Use pd.cut on the "Total Students" column to create a new "School Size" column
per_school_summary['School Size'] = pd.cut(total_students, bins=size_bins, labels=labels)

# Remove formatting from numeric columns for aggregation
numeric_summary = pd.DataFrame({
    'Average Math Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Math': percent_passing_math,
    '% Passing Reading': percent_passing_reading,
    '% Overall Passing': percent_overall_passing
})

# Add the "School Size" column to numeric_summary
numeric_summary['School Size'] = pd.cut(total_students, bins=size_bins, labels=labels)

# Group by "School Size" and calculate mean values
size_summary = numeric_summary.groupby('School Size').mean()

# Format the results
size_summary['Average Math Score'] = size_summary['Average Math Score'].map('{:.2f}'.format)
size_summary['Average Reading Score'] = size_summary['Average Reading Score'].map('{:.2f}'.format)
size_summary['% Passing Math'] = size_summary['% Passing Math'].map('{:.2f}%'.format)
size_summary['% Passing Reading'] = size_summary['% Passing Reading'].map('{:.2f}%'.format)
size_summary['% Overall Passing'] = size_summary['% Overall Passing'].map('{:.2f}%'.format)

# Print the size summary using tabulate
print(f"\033[1mScores by School Size:\033[0m")
print(tabulate(size_summary, headers='keys', tablefmt='fancy_grid'))

Scores by School Size:
╒════════════════════╤══════════════════════╤═════════════════════════╤══════════════════╤═════════════════════╤═════════════════════╕
│ School Size        │   Average Math Score │   Average Reading Score │ % Passing Math   │ % Passing Reading   │ % Overall Passing   │
╞════════════════════╪══════════════════════╪═════════════════════════╪══════════════════╪═════════════════════╪═════════════════════╡
│ Small (<1000)      │                83.82 │                   83.93 │ 93.55%           │ 96.10%              │ 89.88%              │
├────────────────────┼──────────────────────┼─────────────────────────┼──────────────────┼─────────────────────┼─────────────────────┤
│ Medium (1000-2000) │                83.37 │                   83.86 │ 93.60%           │ 96.79%              │ 90.62%              │
├────────────────────┼──────────────────────┼─────────────────────────┼──────────────────┼─────────────────────┼─────────────────────┤
│ Large (2000-5000)  │          

In [24]:
# Remove formatting to convert columns back to numeric for aggregation
per_school_summary['Average Math Score'] = per_school_summary['Average Math Score'].str.rstrip('%').astype('float')
per_school_summary['Average Reading Score'] = per_school_summary['Average Reading Score'].str.rstrip('%').astype('float')
per_school_summary['% Passing Math'] = per_school_summary['% Passing Math'].str.rstrip('%').astype('float')
per_school_summary['% Passing Reading'] = per_school_summary['% Passing Reading'].str.rstrip('%').astype('float')
per_school_summary['% Overall Passing'] = per_school_summary['% Overall Passing'].str.rstrip('%').astype('float')

# Group by "School Type" and calculate mean values
type_summary = per_school_summary.groupby('School Type').mean()

# Format the results
type_summary['Average Math Score'] = type_summary['Average Math Score'].map('{:.2f}%'.format)
type_summary['Average Reading Score'] = type_summary['Average Reading Score'].map('{:.2f}%'.format)
type_summary['% Passing Math'] = type_summary['% Passing Math'].map('{:.2f}%'.format)
type_summary['% Passing Reading'] = type_summary['% Passing Reading'].map('{:.2f}%'.format)
type_summary['% Overall Passing'] = type_summary['% Overall Passing'].map('{:.2f}%'.format)

# Print the type summary using tabulate
print(f"\033[1mScores by School Type:\033[0m")
print(tabulate(type_summary, headers='keys', tablefmt='fancy_grid'))

Scores by School Type:
╒═══════════════╤══════════════════╤══════════════════════╤═════════════════════════╤══════════════════╤═════════════════════╤═════════════════════╕
│ School Type   │   Total Students │ Average Math Score   │ Average Reading Score   │ % Passing Math   │ % Passing Reading   │ % Overall Passing   │
╞═══════════════╪══════════════════╪══════════════════════╪═════════════════════════╪══════════════════╪═════════════════════╪═════════════════════╡
│ Charter       │          1524.25 │ 83.47%               │ 83.90%                  │ 93.62%           │ 96.59%              │ 90.43%              │
├───────────────┼──────────────────┼──────────────────────┼─────────────────────────┼──────────────────┼─────────────────────┼─────────────────────┤
│ District      │          3853.71 │ 76.96%               │ 80.97%                  │ 66.55%           │ 80.80%              │ 53.67%              │
╘═══════════════╧══════════════════╧══════════════════════╧════════════════════════